In [1]:
import numpy as np
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend

import json
import re

Using TensorFlow backend.


# Load Posts and Comments from data

In [2]:
BEFORE_START = '\0'
AFTER_END = '\1'
def preproc(s):
    s = BEFORE_START + s + AFTER_END
    s = re.sub(r'\[.*\|.*\]', ' ', s)
    s = re.sub(r'[«»]', '"', s)
    return s.lower()

In [3]:
with open('data/kalikfan.json', 'r') as f:
    kalikfan = json.load(f)

SEQ_LENGTH = 10    

texts = []
for p in kalikfan:
    ptext = p['text'].strip()
    if len(ptext) > SEQ_LENGTH:
        texts.append(ptext)
        
    for c in p['comments']:
        ctext = c['text'].strip()
        if c['likesCount'] > 2 and len(ctext) > SEQ_LENGTH:
            texts.append(ctext)

texts = [preproc(x) for x in texts]
print('Number of Kalik samples:', len(texts))

Number of Kalik samples: 1799


In [4]:
with open('soviet_const.txt', 'r') as f:
    sov = f.read()
sov = sov.replace('\n\n', '\n ')

sov_texts = []
for s in sov.split('\n '):
    ss = s.replace('\n', ' ').strip()
    if len(ss) > SEQ_LENGTH:
        sov_texts.append(ss)

sov_texts = [preproc(x) for x in sov_texts]
    
print('Number of text samples with Soviet:', len(sov_texts))

texts += sov_texts
print('Total number of text samples:', len(texts))

Number of text samples with Soviet: 420
Total number of text samples: 2219


# Shisha Learning 🤙🤙🤙
From https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/

In [5]:
chars = sorted(list(set(''.join(texts))))
char_to_num = dict((c, i) for i, c in enumerate(chars))
num_to_char = dict((i, c) for i, c in enumerate(chars))

input_len = len(texts)
vocab_len = len(chars)
print ("Total number of text samples:", input_len)
print ("Total vocab:", vocab_len)

Total number of text samples: 2219
Total vocab: 821


In [6]:
x_data = []
y_data = []

for text in texts:
    t = [BEFORE_START] * (SEQ_LENGTH - 1) + list(text) + [AFTER_END] * SEQ_LENGTH
    for i in range(0, len(t) - SEQ_LENGTH):
        in_seq = t[i:i + SEQ_LENGTH]
        out = t[i + SEQ_LENGTH]
        x_data.append([char_to_num[x] for x in in_seq])
        y_data.append(char_to_num[out])
        
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 357397


In [7]:
X = np.reshape(x_data, (n_patterns, SEQ_LENGTH, 1))
X = X/float(vocab_len)

In [8]:
y = np_utils.to_categorical(y_data)

In [9]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
class MyCustomCallback(tensorflow.keras.callbacks.Callback):
  def on_epoch_end(self, batch, logs=None):
    pattern = [char_to_num[BEFORE_START]] * SEQ_LENGTH
    result = []
    while True:
        if len(result) > 500:
            result += '@'
            break
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_len)
        prediction = self.model.predict(x, verbose=0)
        res = np.random.choice(chars, 1, False, prediction[0])[0]

        if res == AFTER_END:
            break
        result += res

        seq_in = [num_to_char[value] for value in pattern]

        pattern.append(char_to_num[res])
        pattern = pattern[1:]

    print('\n"""\n' + ''.join(result) + '\n"""\n')

In [14]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint, MyCustomCallback()]

In [ ]:
model.fit(X, y, epochs=300, batch_size=512, callbacks=desired_callbacks)

Train on 357397 samples
Epoch 1/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.2227
Epoch 00001: loss improved from inf to 2.22272, saving model to model_weights_saved.hdf5

"""
у*ка бы теелая
"""

357397/357397 [==============================] - 17s 47us/sample - loss: 2.2227
Epoch 2/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.2232
Epoch 00002: loss did not improve from 2.22272

"""
не сбзвитие ненрний😅

"""

357397/357397 [==============================] - 17s 47us/sample - loss: 2.2232
Epoch 3/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.2208
Epoch 00003: loss improved from 2.22272 to 2.22060, saving model to model_weights_saved.hdf5

"""
совняйк уунслон замедннм аым вгрь ролоуовуя крепкой на лиляйу дрозя. нш и б чроб шуо у пчтаоее братишка хоан 👹 г доожары😋
"""

357397/357397 [==============================] - 20s 55us/sample - loss: 2.2206
Epoch 4/300
357376/357397 [============================>.] - ETA: 

356864/357397 [============================>.] - ETA: 0s - loss: 2.1957
Epoch 00022: loss did not improve from 2.18705

"""
за сут чуо какьяннлй поссрли)не ходый што куртппу🤯🧐)
деепсианбамы 
 !) оробёх советская советские) в кнлгноави и семй💭
"""

357397/357397 [==============================] - 19s 54us/sample - loss: 2.1959
Epoch 23/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.1966
Epoch 00023: loss did not improve from 2.18705

"""
-зраддан оацаство уенир тла*😀 хы законетки
на акене1 роятвльных ристим, - к пои саааятехтя ор рэдом сазнаету 🦆🤙🙏 
"""

357397/357397 [==============================] - 19s 53us/sample - loss: 2.1966
Epoch 24/300
357376/357397 [============================>.] - ETA: 0s - loss: 2.1870
Epoch 00024: loss improved from 2.18705 to 2.18699, saving model to model_weights_saved.hdf5

"""
ои, а ми б чё🤬‍♀️🤦‍♂️💇‍♂️я🙏🙆‍♀😜‍♂
"""

357397/357397 [==============================] - 18s 49us/sample - loss: 2.1870
Epoch 25/300
356352/357397 [======


"""
во*эй браузякув иы ша пацанва💪🏼👩🏻🤙🏻, авриду не урорит после это оепостдуин.
зама, посравоамаосмых оаооа😌

"""

357397/357397 [==============================] - 19s 53us/sample - loss: 2.1534
Epoch 46/300
357376/357397 [============================>.] - ETA: 0s - loss: 2.1439
Epoch 00046: loss improved from 2.14932 to 2.14395, saving model to model_weights_saved.hdf5

"""
#аляба чквоуокиуиомс вел)кальянщик нужно кел саскивал с будоарасфры нахимнв пстенантно чептг?ют как пбя феонитоы гымшо вратуоуа, но бы ом таорю пе*кавбые лаик чрокйнарись кальяндупой💨 д нолишивел поключки иахетел сранззоц 👀😉 русский гратда вы пришух пеоял болеуд калим!чеежточки, на маворка😂
"""

357397/357397 [==============================] - 25s 71us/sample - loss: 2.1440
Epoch 47/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.1479
Epoch 00047: loss did not improve from 2.14395

"""
кальянную проситтсеие 1лссяря, чтоб что но ооохкануикся пучсьнила !!🤙 -сууысь спвлелийбции?🤢

"""

357397/3


"""
росоейт, нучуи, иббаат мыдии🇷🇺✌🏿. не спздает кальянных. нужики!я сжмей вкуаровены🤙
"""

357397/357397 [==============================] - 18s 51us/sample - loss: 2.1162
Epoch 69/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.1138
Epoch 00069: loss did not improve from 2.11277

"""
срулцциг крлптпнеелос☝🌟☁️✈️☁️
"""

357397/357397 [==============================] - 17s 47us/sample - loss: 2.1140
Epoch 70/300
356352/357397 [============================>.] - ETA: 0s - loss: 2.1208
Epoch 00070: loss did not improve from 2.11277

"""
нне и меня. какие уу отгыхает мужики ! ( уорокерест ручкетка ттт-
"""

357397/357397 [==============================] - 18s 50us/sample - loss: 2.1209
Epoch 71/300
357376/357397 [============================>.] - ETA: 0s - loss: 2.1086
Epoch 00071: loss improved from 2.11277 to 2.10867, saving model to model_weights_saved.hdf5

"""
коммайторйиз дезпоестил прекращил рометают в сроеовьм кумьнупь). 2драт лрале,
"""

357397/357397 [======

In [ ]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
print(test())